In [2]:
import os
import numpy as np
import pandas as pd
import zipfile
import pandas as pd
import geopandas as gpd
from dbfread import DBF
from urllib.request import urlretrieve
from urllib.error import HTTPError
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

In [3]:
#Go back one level from notebook to data 
output_relative_dir = '../data/'

if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)
    
for target_dir in ('curated','outer'):
    if not os.path.exists(output_relative_dir + target_dir):
        os.makedirs(output_relative_dir + target_dir)

### Download census data

In [4]:
url = "https://www.abs.gov.au/census/find-census-data/datapacks/download/2021_GCP_SA2_for_AUS_short-header.zip"#year-month.parquet
output_dir = "../data/outer/census.zip"
# download
urlretrieve(url, output_dir) 
print("complete")

complete


In [5]:
files = zipfile.ZipFile('../data/outer/census.zip','r')

for file in files.namelist():
    files.extract(file, f"../data/outer/census")

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/09/18 05:37:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/18 05:37:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/18 05:37:38 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [7]:
types = ["A", "B"]
for type in types:
    G04= spark.read.option("header",True).csv(f'../data/outer/census/2021 Census GCP Statistical Area 2 for AUS/2021Census_G04{type}_AUST_SA2.csv') #read the parquet 
    G04 = G04.na.drop(subset=G04.columns)
    G04.write.mode("overwrite").parquet(f"../data/curated/census_data_2021_04{type}.parquet")


22/09/15 12:35:13 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


AnalysisException: path file:/Users/seren/Downloads/project_2/generic-buy-now-pay-later-project-group-16-project2-fighting-group/data/curated/census_data_2021_04A.parquet already exists.

In [8]:
types = ["A", "B", "C"]
for type in types:
    G17= spark.read.option("header",True).csv(f'../data/outer/census/2021 Census GCP Statistical Area 2 for AUS/2021Census_G17{type}_AUST_SA2.csv') #read the parquet 
    G17 = G17.na.drop(subset=G17.columns)

    G17.write.mode("overwrite").parquet(f"../data/curated/census_data_2021_17{type}.parquet")

KeyboardInterrupt: 

### Download postcode and SA2 data

In [5]:
url = "http://github.com/matthewproctor/australianpostcodes/zipball/master"
output_dir = "../data/outer/total_pto2.zip"
# download
urlretrieve(url, output_dir) 
print("complete")


KeyboardInterrupt: 

In [ ]:
files = zipfile.ZipFile('../data/outer/total_pto2.zip','r')
for file in files.namelist():
    files.extract(file, f"../data/outer/total_pto2")

In [ ]:
post_sa2= spark.read.option("header",True).csv('../data/outer/total_pto2/matthewproctor-australianpostcodes-6f8a994/australian_postcodes.csv') 
print(post_sa2.dtypes)
post_sa2 = post_sa2.select("id", "postcode", "Lat_precise", "Long_precise", "SA2_MAINCODE_2016")
print(post_sa2.count() - post_sa2.na.drop().count())
post_sa2 = post_sa2.na.drop()
#177 rows with nulls dropped

#get the average latitude and longitude of the postcode if it have duplicates
post_sa2 = post_sa2\
    .withColumn("Lat_double", post_sa2["Lat_precise"].cast("double"))\
    .withColumn("Long_double", post_sa2["Long_precise"].cast("double"))
post_sa2 = post_sa2\
    .groupBy("postcode", "SA2_MAINCODE_2016")\
    .agg(F.avg("Lat_double").alias("avg_lat"), F.avg("Long_double").alias("avg_long"))

post_sa2.write.mode("overwrite").parquet("../data/curated/postcode_SA2.parquet")

[('id', 'string'), ('postcode', 'string'), ('locality', 'string'), ('state', 'string'), ('long', 'string'), ('lat', 'string'), ('dc', 'string'), ('type', 'string'), ('status', 'string'), ('sa3', 'string'), ('sa3name', 'string'), ('sa4', 'string'), ('sa4name', 'string'), ('region', 'string'), ('Lat_precise', 'string'), ('Long_precise', 'string'), ('SA1_MAINCODE_2011', 'string'), ('SA1_MAINCODE_2016', 'string'), ('SA2_MAINCODE_2016', 'string'), ('SA2_NAME_2016', 'string'), ('SA3_CODE_2016', 'string'), ('SA3_NAME_2016', 'string'), ('SA4_CODE_2016', 'string'), ('SA4_NAME_2016', 'string'), ('RA_2011', 'string'), ('RA_2016', 'string'), ('MMM_2015', 'string'), ('MMM_2019', 'string'), ('ced', 'string'), ('altitude', 'string'), ('chargezone', 'string'), ('phn_code', 'string'), ('phn_name', 'string'), ('lgaregion', 'string'), ('electorate', 'string'), ('electoraterating', 'string')]
177


### Download SA2 2021 information

In [6]:
url = "https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/SA2_2021_AUST_SHP_GDA2020.zip" 
#year-month.parquet
output_dir = "../data/outer/2021sa2_shapefile.zip"
# download
urlretrieve(url, output_dir) 
print("complete")

complete


In [7]:
files = zipfile.ZipFile('../data/outer/2021sa2_shapefile.zip','r')
for file in files.namelist():
    files.extract(file, f"../data/outer/2021sa2_shapefile")

### Download shapefile for each SA2 region
### !!!WARNING!!! THIS CODE IS GOING TO TAKE 10 MINUTES

In [8]:
def get_shapefile(url):
    geojson_option = "?_profile=oai&_mediatype=application/geo+json"
    try:
        shape = str(gpd.read_file(url + geojson_option).iat[0,-1])
    except HTTPError:
        shape = ""
    return shape

get_shapefile_udf = udf(lambda a: get_shapefile(a),StringType())


In [25]:
path = r'../data/outer/2021sa2_shapefile/SA2_2021_AUST_GDA2020.dbf' 
table = DBF(path)
sa2_pd_temp = pd.DataFrame(iter(table))
sa2_2021_temp = spark.createDataFrame(sa2_pd_temp) 

sa2_2021_vic = sa2_2021_temp.filter(F.col("STE_NAME21")=="Victoria")
sa2_2021_vic_w_geo = sa2_2021_vic.withColumn("geometry", get_shapefile_udf(F.col("LOCI_URI21")))
sa2_2021_vic.count()

524

too big can't write to parquet

In [27]:
sa2_2021_vic_w_geo = sa2_2021_vic_w_geo.select("SA2_CODE21", "SA2_NAME21", "geometry")
sa2_2021_vic_w_geo.write.option("maxRecordsPerFile", 20).mode("overwrite").parquet("../data/curated/SA2_2021_VIC_shapefile.parquet")

#sa2_2021_vic_w_geo.write.option("maxRecordsPerFile", 20).mode("overwrite").parquet("../data/curated/SA2_2021_VIC_shapefile.parquet")
#sa2_2021_ = sa2_2021_temp.select("SA2_CODE21")
#sa2_2021_.count

KeyboardInterrupt: 

In [ ]:
sa2_2021_temp(5)

KeyboardInterrupt: 

### Download file with 2016 SA2 info and 2021 SA2 info

In [9]:
url = "https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/correspondences/CG_SA2_2016_SA2_2021.csv"
output_dir = "../data/outer/correspondences.csv"
# download
urlretrieve(url, output_dir) 
print("complete")

complete


In [11]:
correspondences= spark.read.option("header",True).csv('../data/outer/correspondences.csv') #read the parquet 
correspondences = correspondences.na.drop()
correspondences.write.parquet("../data/curated/correspondences.parquet")
correspondences

SA2_MAINCODE_2016,SA2_NAME_2016,SA2_CODE_2021,SA2_NAME_2021,RATIO_FROM_TO,INDIV_TO_REGION_QLTY_INDICATOR,OVERALL_QUALITY_INDICATOR,BMOS_NULL_FLAG
101021007,Braidwood,101021007,Braidwood,1,Good,Good,0
101021008,Karabar,101021008,Karabar,1,Good,Good,0
101021009,Queanbeyan,101021009,Queanbeyan,1,Good,Good,0
101021010,Queanbeyan - East,101021010,Queanbeyan - East,1,Good,Good,0
101021011,Queanbeyan Region,101021610,Googong,0.0967097,Poor,Good,0
101021011,Queanbeyan Region,101021611,Queanbeyan Surrounds,0.9032903,Good,Good,0
101021012,Queanbeyan West -...,101021012,Queanbeyan West -...,1,Good,Good,0
101031013,Bombala,101031013,Bombala,1,Good,Good,0
101031014,Cooma,101031014,Cooma,1,Good,Good,0
101031015,Cooma Region,101031015,Cooma Surrounds,1,Good,Good,0


22/09/15 13:32:46 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 875639 ms exceeds timeout 120000 ms
22/09/15 13:32:46 WARN SparkContext: Killing executors is not supported by current scheduler.


### Find 2016 post code and 2021 sa2 correspondence

In [ ]:
print(post_sa2.dtypes)
print(correspondences.dtypes)
post_sa2_2021 = correspondences.join(post_sa2,correspondences.SA2_MAINCODE_2016 == post_sa2.SA2_MAINCODE_2016,"left") 

[('postcode', 'string'), ('SA2_MAINCODE_2016', 'string'), ('avg_lat', 'double'), ('avg_long', 'double')]
[('SA2_MAINCODE_2016', 'string'), ('SA2_NAME_2016', 'string'), ('SA2_CODE_2021', 'string'), ('SA2_NAME_2021', 'string'), ('RATIO_FROM_TO', 'string'), ('INDIV_TO_REGION_QLTY_INDICATOR', 'string'), ('OVERALL_QUALITY_INDICATOR', 'string'), ('BMOS_NULL_FLAG', 'string')]


## JOIN tables

In [ ]:
post_sa2_2021 = post_sa2_2021.drop("SA2_NAME_2016", "SA2_MAINCODE_2016")
post_sa2_2021.write.mode("overwrite").parquet("../data/curated/postcode_sa2_conrrespondences.parquet")

### Code before

In [ ]:
"""
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "id": "4ee2f758-860f-4f97-89dd-24167d16081b",
   "metadata": {},
   "outputs": [],
   "source": [
    "import os\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import zipfile\n",
    "import pandas as pd\n",
    "import geopandas as gpd\n",
    "from dbfread import DBF\n",
    "from urllib.request import urlretrieve\n",
    "from urllib.error import HTTPError\n",
    "from pyspark.sql.types import StringType\n",
    "from pyspark.sql.functions import udf"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "id": "62f8ed27",
   "metadata": {},
   "outputs": [],
   "source": [
    "#Go back one level from notebook to data \n",
    "output_relative_dir = '../data/'\n",
    "\n",
    "if not os.path.exists(output_relative_dir):\n",
    "    os.makedirs(output_relative_dir)\n",
    "    \n",
    "for target_dir in ('curated','outer'):\n",
    "    if not os.path.exists(output_relative_dir + target_dir):\n",
    "        os.makedirs(output_relative_dir + target_dir)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "50142b46-ba2c-4b18-ad27-154b22342e9e",
   "metadata": {},
   "source": [
    "### Download census data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "id": "ce0340c5",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "complete\n"
     ]
    }
   ],
   "source": [
    "url = \"https://www.abs.gov.au/census/find-census-data/datapacks/download/2021_GCP_SA2_for_AUS_short-header.zip\"#year-month.parquet\n",
    "output_dir = \"../data/outer/census.zip\"\n",
    "# download\n",
    "urlretrieve(url, output_dir) \n",
    "print(\"complete\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "id": "e098cb87",
   "metadata": {},
   "outputs": [],
   "source": [
    "files = zipfile.ZipFile('../data/outer/census.zip','r')\n",
    "\n",
    "for file in files.namelist():\n",
    "    files.extract(file, f\"../data/outer/census\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "33a4c2e6-45d2-4763-afb3-3e1a39e591dd",
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "id": "5da580d7",
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "22/09/12 13:02:48 WARN Utils: Your hostname, SerennoMacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.13.245.25 instead (on interface en0)\n",
      "22/09/12 13:02:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address\n",
      "22/09/12 13:02:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable\n",
      "Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties\n",
      "Setting default log level to \"WARN\".\n",
      "To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).\n"
     ]
    }
   ],
   "source": [
    "from pyspark.sql import SparkSession\n",
    "from pyspark.sql import functions as F\n",
    "# Create a spark session (which will run spark jobs)\n",
    "spark = (\n",
    "    SparkSession.builder.appName(\"MAST30034 Tutorial 1\")\n",
    "    .config(\"spark.sql.repl.eagerEval.enabled\", True) \n",
    "    .config(\"spark.sql.parquet.cacheMetadata\", \"true\")\n",
    "    .config(\"spark.sql.session.timeZone\", \"Etc/UTC\")\n",
    "    .getOrCreate()\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "dc2ef49a",
   "metadata": {},
   "outputs": [],
   "source": [
    "types = [\"A\", \"B\"]\n",
    "for type in types:\n",
    "    G04= spark.read.option(\"header\",True).csv(f'../data/outer/census/2021 Census GCP Statistical Area 2 for AUS/2021Census_G04{type}_AUST_SA2.csv') #read the parquet \n",
    "    G04 = G04.na.drop(subset=G04.columns)\n",
    "    G04.write.parquet(f\"../data/curated/census_data_2021_04{type}.parquet\")\n"
   ]
  },
  {
   "cell_type": "code",
<<<<<<< Updated upstream
=======
<<<<<<< HEAD
   "execution_count": null,
   "id": "bd4d074c-b318-47d2-90b2-7c6d997e0fde",
   "metadata": {},
   "outputs": [],
=======
>>>>>>> Stashed changes
   "execution_count": 92,
   "id": "bd4d074c-b318-47d2-90b2-7c6d997e0fde",
   "metadata": {},
   "outputs": [
    {
     "ename": "TypeError",
     "evalue": "'str' object is not callable",
     "output_type": "error",
     "traceback": [
      "\u001b[0;31m---------------------------------------------------------------------------\u001b[0m",
      "\u001b[0;31mTypeError\u001b[0m                                 Traceback (most recent call last)",
      "\u001b[0;32m/var/folders/_r/5qlbmxqj2yscq5d1p31h95m80000gn/T/ipykernel_55661/1206215559.py\u001b[0m in \u001b[0;36m<module>\u001b[0;34m\u001b[0m\n\u001b[1;32m      3\u001b[0m     \u001b[0mG17\u001b[0m\u001b[0;34m=\u001b[0m \u001b[0mspark\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mread\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0moption\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m\"header\"\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;32mTrue\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mcsv\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34mf'../data/outer/census/2021 Census GCP Statistical Area 2 for AUS/2021Census_G17{type}_AUST_SA2.csv'\u001b[0m\u001b[0;34m)\u001b[0m \u001b[0;31m#read the parquet\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m      4\u001b[0m     \u001b[0mG17\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mG17\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mna\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mdrop\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0msubset\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0mG17\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mcolumns\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m----> 5\u001b[0;31m     \u001b[0mprint\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mtype\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mG17\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m      6\u001b[0m     \u001b[0mG17\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mwrite\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mmode\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m\"overwrite\"\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mparquet\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34mf\"../data/curated/census_data_2021_17{type}.parquet\"\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
      "\u001b[0;31mTypeError\u001b[0m: 'str' object is not callable"
     ]
    }
   ],
>>>>>>> c46ee70c8914e4763bcac54c47d8249692516577
   "source": [
    "types = [\"A\", \"B\", \"C\"]\n",
    "for type in types:\n",
    "    G17= spark.read.option(\"header\",True).csv(f'../data/outer/census/2021 Census GCP Statistical Area 2 for AUS/2021Census_G17{type}_AUST_SA2.csv') #read the parquet \n",
    "    G17 = G17.na.drop(subset=G17.columns)\n",
    "\n",
    "    G17.write.mode(\"overwrite\").parquet(f\"../data/curated/census_data_2021_17{type}.parquet\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "0973831e",
   "metadata": {},
   "outputs": [],
   "source": [
    "url = \"http://github.com/matthewproctor/australianpostcodes/zipball/master\"#year-month.parquet\n",
    "output_dir = \"../data/outer/total_pto2.zip\"\n",
    "# download\n",
    "urlretrieve(url, output_dir) \n",
    "print(\"complete\")\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "da09bc21",
   "metadata": {},
   "outputs": [],
   "source": [
    "files = zipfile.ZipFile('../data/outer/total_pto2.zip','r')\n",
    "for file in files.namelist():\n",
    "    files.extract(file, f\"../data/outer/total_pto2\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "d74e0c8c",
   "metadata": {},
   "source": [
    "### Download SA2 Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "aa60ad1f",
   "metadata": {},
   "outputs": [],
   "source": [
    "url = \"https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/SA2_2021_AUST_SHP_GDA2020.zip\" \n",
    "#year-month.parquet\n",
    "output_dir = \"../data/outer/2021sa2_shapefile.zip\"\n",
    "# download\n",
    "urlretrieve(url, output_dir) \n",
    "print(\"complete\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "acf29ca9",
   "metadata": {},
   "outputs": [],
   "source": [
    "files = zipfile.ZipFile('../data/outer/2021sa2_shapefile.zip','r')\n",
    "for file in files.namelist():\n",
    "    files.extract(file, f\"../data/outer/2021sa2_shapefile\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "7d2ad5fc",
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "markdown",
   "id": "8a4f4f31",
   "metadata": {},
   "source": [
    "### Download shapefile for each SA2 region\n",
    "### !!!WARNING!!! THIS CODE IS GOING TO TAKE 30 MINUTES"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "id": "bc4c7c73",
   "metadata": {},
   "outputs": [],
   "source": [
    "def get_shapefile(url):\n",
    "    geojson_option = \"?_profile=oai&_mediatype=application/geo+json\"\n",
    "    try:\n",
    "        shape = str(gpd.read_file(url + geojson_option).iat[0,-1])\n",
    "    except HTTPError:\n",
    "        shape = \"\"\n",
    "    return shape\n",
    "    \n",
    "get_shapefile_udf = udf(lambda a: get_shapefile(a),StringType())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "id": "405d8b35",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "done\n"
     ]
    }
   ],
   "source": [
    "### Download SA2 2021 information\n",
    "path = r'../data/outer/2021sa2_shapefile/SA2_2021_AUST_GDA2020.dbf' # 文件目录\n",
    "table = DBF(path)\n",
    "geojson_option = \"?_profile=oai&_mediatype=application/geo+json\"\n",
    "sa2_pd_temp = pd.DataFrame(iter(table))\n",
    "sa2_2021_temp = spark.createDataFrame(sa2_pd_temp) \n",
    "print(\"done\")\n",
    "sa2_2021_temp = sa2_2021_temp.withColumn(\"geometry\", get_shapefile_udf(F.col(\"LOCI_URI21\")))\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 36,
   "id": "4fd35e5b",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "+----------------------+\n",
      "|SA2_NAME21            |\n",
      "+----------------------+\n",
      "|Reservoir - North East|\n",
      "|Reservoir - North West|\n",
      "|Reservoir - South East|\n",
      "|Reservoir - South West|\n",
      "+----------------------+\n",
      "\n"
     ]
    }
   ],
   "source": [
    "sa2_2021_temp.filter(F.col(\"SA2_NAME21\").contains(\"Reservoir \")).select(\"SA2_NAME21\").show(truncate = False)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "fc4df311-55d0-4b69-a8db-3ed1eea91a2c",
   "metadata": {},
   "outputs": [],
   "source": [
    "sa2_2021_selected = sa2_2021_temp.select(\"SA2_CODE21\", \"SA2_NAME21\", \"geometry\")\n",
    "sa2_2021_selected.to_parquet(\"../data/curated/SA2_2021_AUST_GDA2020.parquet\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "cc7231f3",
   "metadata": {},
   "source": [
    "###  Download postal area shapefile"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "id": "129155d5",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "complete\n"
     ]
    }
   ],
   "source": [
    "url = \"https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/POA_2021_AUST_GDA2020_SHP.zip\"\n",
    "\n",
    "#year-month.parquet\n",
    "output_dir = \"../data/outer/2021poa_shapefile.zip\"\n",
    "# download\n",
    "urlretrieve(url, output_dir) \n",
    "print(\"complete\")\n",
    "\n",
    "files = zipfile.ZipFile('../data/outer/2021poa_shapefile.zip','r')\n",
    "for file in files.namelist():\n",
    "    files.extract(file, f\"../data/outer/2021poa_shapefile\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "fca55e33",
   "metadata": {},
   "source": [
    "### Download shapefile for each POA region\n",
    "### !!!WARNING!!! THIS CODE IS GOING TO TAKE 10 MINUTES"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "id": "74478f35",
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "                                                                                \r"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "+----------+----------+----------+----------+----------+--------------------+---------------+----------------+\n",
      "|POA_CODE21|POA_NAME21|AUS_CODE21|AUS_NAME21|AREASQKM21|          LOCI_URI21|     SHAPE_Leng|      SHAPE_Area|\n",
      "+----------+----------+----------+----------+----------+--------------------+---------------+----------------+\n",
      "|      0800|      0800|       AUS| Australia|    3.1731|http://linked.dat...|0.0818928754197|2.63898689344E-4|\n",
      "|      0810|      0810|       AUS| Australia|   24.4283|http://linked.dat...| 0.241859199671|0.00203051956323|\n",
      "|      0812|      0812|       AUS| Australia|   35.8899|http://linked.dat...| 0.278788498785|0.00298326491157|\n",
      "|      0820|      0820|       AUS| Australia|   39.0642|http://linked.dat...|  0.40913400972|0.00324767063304|\n",
      "|      0822|      0822|       AUS| Australia|150775.803|http://linked.dat...|  90.6018310423|   12.5642382212|\n",
      "|      0828|      0828|       AUS| Australia|   28.6532|http://linked.dat...| 0.246298859951|0.00238217099071|\n",
      "|      0829|      0829|       AUS| Australia|   44.8402|http://linked.dat...| 0.306927000606|  0.003728299776|\n",
      "|      0830|      0830|       AUS| Australia|   235.004|http://linked.dat...|  1.38498281675|   0.01953482157|\n",
      "|      0832|      0832|       AUS| Australia|   19.7272|http://linked.dat...| 0.196642713629|0.00164055074459|\n",
      "|      0834|      0834|       AUS| Australia|   32.6302|http://linked.dat...| 0.261017010952|0.00271402831805|\n",
      "+----------+----------+----------+----------+----------+--------------------+---------------+----------------+\n",
      "only showing top 10 rows\n",
      "\n",
      "done\n"
     ]
    }
   ],
   "source": [
    "### Download SA2 2021 information\n",
    "path = r'../data/outer/2021poa_shapefile/POA_2021_AUST_GDA2020.dbf' # 文件目录\n",
    "table = DBF(path)\n",
    "pd_temp = pd.DataFrame(iter(table))\n",
    "poa_temp = spark.createDataFrame(pd_temp) \n",
    "poa_temp.show(10)\n",
    "print(\"done\")\n",
    "poa_temp = poa_temp.withColumn(\"geometry\", get_shapefile_udf(F.col(\"LOCI_URI21\")))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "id": "7a7255b3",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "done\n"
     ]
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "[Stage 1:>                                                          (0 + 4) / 4]\r"
     ]
    },
    {
     "ename": "KeyboardInterrupt",
     "evalue": "",
     "output_type": "error",
     "traceback": [
      "\u001b[0;31m---------------------------------------------------------------------------\u001b[0m",
      "\u001b[0;31mKeyboardInterrupt\u001b[0m                         Traceback (most recent call last)",
      "\u001b[0;32m/var/folders/_r/5qlbmxqj2yscq5d1p31h95m80000gn/T/ipykernel_4327/1105041411.py\u001b[0m in \u001b[0;36m<module>\u001b[0;34m\u001b[0m\n\u001b[1;32m      2\u001b[0m \u001b[0mprint\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m\"done\"\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m      3\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m----> 4\u001b[0;31m \u001b[0mpoa_2021_selected\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mwrite\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mpartitionBy\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m\"POA_CODE21\"\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;34m\"geometry\"\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mmode\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m'overwrite'\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mparquet\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m\"../data/curated/POA_2021_AUST_GDA2020.parquet\"\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m",
      "\u001b[0;32m~/opt/anaconda3/lib/python3.7/site-packages/pyspark/sql/readwriter.py\u001b[0m in \u001b[0;36mparquet\u001b[0;34m(self, path, mode, partitionBy, compression)\u001b[0m\n\u001b[1;32m   1248\u001b[0m             \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mpartitionBy\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mpartitionBy\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   1249\u001b[0m         \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_set_opts\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mcompression\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0mcompression\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m-> 1250\u001b[0;31m         \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_jwrite\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mparquet\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mpath\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m   1251\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   1252\u001b[0m     \u001b[0;32mdef\u001b[0m \u001b[0mtext\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mself\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mpath\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mcompression\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0;32mNone\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mlineSep\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0;32mNone\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
      "\u001b[0;32m~/opt/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py\u001b[0m in \u001b[0;36m__call__\u001b[0;34m(self, *args)\u001b[0m\n\u001b[1;32m   1306\u001b[0m             \u001b[0mproto\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mEND_COMMAND_PART\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   1307\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m-> 1308\u001b[0;31m         \u001b[0manswer\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mgateway_client\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0msend_command\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mcommand\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m   1309\u001b[0m         return_value = get_return_value(\n\u001b[1;32m   1310\u001b[0m             answer, self.gateway_client, self.target_id, self.name)\n",
      "\u001b[0;32m~/opt/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py\u001b[0m in \u001b[0;36msend_command\u001b[0;34m(self, command, retry, binary)\u001b[0m\n\u001b[1;32m   1036\u001b[0m         \u001b[0mconnection\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_get_connection\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   1037\u001b[0m         \u001b[0;32mtry\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m-> 1038\u001b[0;31m             \u001b[0mresponse\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mconnection\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0msend_command\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mcommand\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m   1039\u001b[0m             \u001b[0;32mif\u001b[0m \u001b[0mbinary\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   1040\u001b[0m                 \u001b[0;32mreturn\u001b[0m \u001b[0mresponse\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_create_connection_guard\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mconnection\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
      "\u001b[0;32m~/opt/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py\u001b[0m in \u001b[0;36msend_command\u001b[0;34m(self, command)\u001b[0m\n\u001b[1;32m   1203\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   1204\u001b[0m         \u001b[0;32mtry\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m-> 1205\u001b[0;31m             \u001b[0manswer\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0msmart_decode\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mstream\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mreadline\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m[\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m-\u001b[0m\u001b[0;36m1\u001b[0m\u001b[0;34m]\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m   1206\u001b[0m             \u001b[0mlogger\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mdebug\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m\"Answer received: {0}\"\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mformat\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0manswer\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   1207\u001b[0m             \u001b[0;32mif\u001b[0m \u001b[0manswer\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mstartswith\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mproto\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mRETURN_MESSAGE\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
      "\u001b[0;32m~/opt/anaconda3/lib/python3.7/socket.py\u001b[0m in \u001b[0;36mreadinto\u001b[0;34m(self, b)\u001b[0m\n\u001b[1;32m    587\u001b[0m         \u001b[0;32mwhile\u001b[0m \u001b[0;32mTrue\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    588\u001b[0m             \u001b[0;32mtry\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 589\u001b[0;31m                 \u001b[0;32mreturn\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_sock\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mrecv_into\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mb\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    590\u001b[0m             \u001b[0;32mexcept\u001b[0m \u001b[0mtimeout\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    591\u001b[0m                 \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_timeout_occurred\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0;32mTrue\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
      "\u001b[0;31mKeyboardInterrupt\u001b[0m: "
     ]
    }
   ],
   "source": [
    "#poa_2021_selected = poa_temp.select(\"POA_CODE21\", \"AUS_CODE21\", \"geometry\")\n",
    "#print(\"done\")\n",
    "\n",
    "#poa_2021_selected.write.partitionBy(\"POA_CODE21\", \"geometry\").mode('overwrite').parquet(\"../data/curated/POA_2021_AUST_GDA2020.parquet\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "10b69bec",
   "metadata": {},
   "source": [
    "### Connect SA2 and POA "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "865fbf69",
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "markdown",
   "id": "607fedb4-d49f-484c-9a0b-8a53203f5c8f",
   "metadata": {},
   "source": [
    "# ------------------------NOT USING ANYMORE -------------------------\n",
    "### Download postcode and SA2 data"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "6eb33f0b-4040-4f05-bca6-f32ade4d79e2",
   "metadata": {},
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "cc2855ee",
   "metadata": {},
   "outputs": [],
   "source": [
    "post_sa2= spark.read.option(\"header\",True).csv('../data/outer/total_pto2/matthewproctor-australianpostcodes-6f8a994/australian_postcodes.csv') \n",
    "print(post_sa2.dtypes)\n",
    "post_sa2 = post_sa2.select(\"id\", \"postcode\", \"Lat_precise\", \"Long_precise\", \"SA2_MAINCODE_2016\")\n",
    "print(post_sa2.count() - post_sa2.na.drop().count())\n",
    "post_sa2 = post_sa2.na.drop()\n",
    "#177 rows with nulls dropped\n",
    "\n",
    "#get the average latitude and longitude of the postcode if it have duplicates\n",
    "post_sa2 = post_sa2\\\n",
    "    .withColumn(\"Lat_double\", post_sa2[\"Lat_precise\"].cast(\"double\"))\\\n",
    "    .withColumn(\"Long_double\", post_sa2[\"Long_precise\"].cast(\"double\"))\n",
    "post_sa2 = post_sa2\\\n",
    "    .groupBy(\"postcode\", \"SA2_MAINCODE_2016\")\\\n",
    "    .agg(F.avg(\"Lat_double\").alias(\"avg_lat\"), F.avg(\"Long_double\").alias(\"avg_long\"))\n",
    "\n",
    "post_sa2.write.parquet(\"../data/curated/postcode_SA2.parquet\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "9e75fd20-c249-4d2e-9eb5-48e74bb8d084",
   "metadata": {},
   "outputs": [],
   "source": [
    "sa2_2021_temp"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "195196c0-f0b8-423f-b663-08b925186ab3",
   "metadata": {},
   "source": [
    "### Download file with 2016 SA2 info and 2021 SA2 info"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "368d62a4",
   "metadata": {},
   "outputs": [],
   "source": [
    "url = \"https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/correspondences/CG_SA2_2016_SA2_2021.csv\"\n",
    "output_dir = \"../data/outer/correspondences.csv\"\n",
    "# download\n",
    "urlretrieve(url, output_dir) \n",
    "print(\"complete\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "f2b15f51",
   "metadata": {},
   "outputs": [],
   "source": [
    "correspondences= spark.read.option(\"header\",True).csv('../data/outer/correspondences.csv') #read the parquet \n",
    "correspondences = correspondences.na.drop()\n",
    "correspondences.write.parquet(\"../data/curated/correspondences.parquet\")\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "65a43a9f-cea5-4924-8a01-af53f1944691",
   "metadata": {},
   "outputs": [],
   "source": [
    "shapes_list = []\n",
    "for row in range(len(poa_temp)):\n",
    "    url = poa_temp.iat[row,5]\n",
    "    try:\n",
    "        shape = str(gpd.read_file(url + geojson_option).iat[0,-1])\n",
    "    except HTTPError:\n",
    "        shape = \"\"\n",
    "    shapes_list.append(shape)\n",
    "\n",
    "poa_temp[\"geometry\"] = shapes_list"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3.7.13 ('base')",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.13"
  },
  "vscode": {
   "interpreter": {
    "hash": "664b9b2cfc08063ed3ab9f92e6d6cbdfe30eb833d966541c89dc480f035db465"
   }
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}"""